In [127]:
from lxml import etree
import urllib3
import requests
import re
import pandas as pd
import numpy as np
import io
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

In [9]:
#date_title = html.xpath('//*[@class="tableheader"]//text()')
# file_name & link (one to one)
#file_name = get_file_info(html,'//*//td[2]//text()')
'''
    headers = {
        "Host": "interface.sina.cn",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0",
        "Accept": "*/*",
        "Accept-Language": "zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": r"http://www.sina.com.cn/mid/search.shtml?range=all&c=news&q=%E6%97%85%E6%B8%B8&from=home&ie=utf-8",
        "Cookie": "ustat=__172.16.93.31_1580710312_0.68442000; genTime=1580710312; vt=99; Apache=9855012519393.69.1585552043971; SINAGLOBAL=9855012519393.69.1585552043971; ULV=1585552043972:1:1:1:9855012519393.69.1585552043971:; historyRecord={'href':'https://news.sina.cn/','refer':'https://sina.cn/'}; SMART=0; dfz_loc=gd-default",
        "TE": "Trailers"
    }
'''

'\n    headers = {\n        "Host": "interface.sina.cn",\n        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0",\n        "Accept": "*/*",\n        "Accept-Language": "zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2",\n        "Accept-Encoding": "gzip, deflate, br",\n        "Connection": "keep-alive",\n        "Referer": r"http://www.sina.com.cn/mid/search.shtml?range=all&c=news&q=%E6%97%85%E6%B8%B8&from=home&ie=utf-8",\n        "Cookie": "ustat=__172.16.93.31_1580710312_0.68442000; genTime=1580710312; vt=99; Apache=9855012519393.69.1585552043971; SINAGLOBAL=9855012519393.69.1585552043971; ULV=1585552043972:1:1:1:9855012519393.69.1585552043971:; historyRecord={\'href\':\'https://news.sina.cn/\',\'refer\':\'https://sina.cn/\'}; SMART=0; dfz_loc=gd-default",\n        "TE": "Trailers"\n    }\n'

In [10]:
header = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36",
    "Connection": "keep-alive",
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7"}

In [11]:
url = "https://www.rbi.org.in/scripts/Annualpolicy.aspx"
def get_html(header,url):
    r = requests.get(url)
    print(r)
    r_text = r.text
    html=etree.HTML(r_text)
    return html

In [12]:
def help_filter(s):
    return re.sub(r'[^a-zA-Z0-9]', '', s)

def get_file_info(html,path):
    file_info = html.xpath(path)

    file_info = list(filter(lambda x: help_filter(x) != "",file_info))
    target = ["Press Release","Full Document"]
    count = 0
    IsPreIn = 0
    new_file_info = []
    while count<len(file_info):
        file_info[count] = file_info[count].strip(" ")
        # cur in target
        if file_info[count] in target:
            # pre not in target
            if IsPreIn==0:
                new_file_info = new_file_info[:-1]
                new_file_info.append(file_info[count-1] +"-"+ file_info[count])
            # pre in target
            else:
                new_file_info.append(file_info[count-2] +"-"+ file_info[count])
        else:
            new_file_info.append(file_info[count])
        
        count+=1
    return new_file_info

In [13]:
def get_file_dict(file_info):
    data_dict = {
        "date":[],
        "title":[],
        "file_name":[]
    }
    date_pos = [i for i,info in enumerate(file_info) if len(info) == 12]
    
    
    for i in range(len(date_pos)-1):
        files_num = (date_pos[i+1]-date_pos[i])-2
        data_dict['date']+= [file_info[date_pos[i]]]*files_num
        data_dict['title']+= [file_info[date_pos[i]+1]]*files_num
        data_dict['file_name']+= file_info[date_pos[i]+2:date_pos[i]+2+files_num]

    files_num = (len(file_info)-date_pos[-1])-2
    data_dict['date']+= [file_info[date_pos[-1]]]*files_num
    data_dict['title']+= [file_info[date_pos[-1]+1]]*files_num
    data_dict['file_name']+= file_info[date_pos[-1]+2:]

    return data_dict

In [16]:
html = get_html(header,url)
file_info = get_file_info(html,'//*[@class="brd-ptable"]//td//text()')
df = pd.DataFrame(get_file_dict(file_info))

link = html.xpath('//*//td//a//@href')
link = link[:-1]
link = link+[None]*(len(df)-len(link))

df['file_link'] = link
df['isScript'] = df['file_link'].apply(lambda s: 0 if (s == None) or ("script" not in s.lower()) else 1)

<Response [200]>


In [36]:
df_script = df[df.isScript == 1].reset_index(drop = True)

# Get Script Content

In [81]:
# Get script content
file_url = df_script['file_link'][0]
file_html = get_html(header,file_url)

<Response [200]>


In [332]:
file_title_info = file_html.xpath('//*[@class="tableheader"]//text()')
file_content = file_html.xpath('//*[@class="tablecontent1"]//p//text()')
pdf_url = file_html.xpath('//*[@class="tableheader"]//a//@href')[0]

In [329]:
def download_pdf(header,save_path,pdf_name,pdf_url):
    response = requests.get(pdf_url, headers=header)
    bytes_io = io.BytesIO(response.content)
    with open(save_path + "%s.PDF" % pdf_name, mode='wb') as f:
        f.write(bytes_io.getvalue())
        print('%s.PDF, Download Successed！' % (pdf_name))

In [336]:
path = r'C://Users//raina//Dropbox//Chengjun//Text Analysis//download_scrips//2021-2022//'
name= df_script['title'][0]
#pdf_url="http://static.cninfo.com.cn/finalpage/2008-03-27/38334292.PDF"
download_pdf(header,path, name, pdf_url = pdf_url )

Minutes of the Monetary Policy Committee.PDF, Download Successed！


# Simulate user click

In [171]:
def get_page(driver,item):
    """
        INPUTS:
            item: onclick item
        RETURNS:
            html object of the web page
    """
    # Waiting for the response
    time.sleep(10)
    driver.find_element(By.ID, item).click()
    return driver.page_source

In [172]:
# Obtain page source and store the html into a dictionary

def get_all_page(url):
    # get year tree and onclick id
    html = get_html(header,url)
    
    # year tree
    year_list = html.xpath('//*[@class="grid_tree archives alpha"]//text()')
    year_list.pop(10)
    
    # onclick_id
    onclick_id = html.xpath('//*[@class="grid_tree archives alpha"]//@id')
    del onclick_id[0]
    del onclick_id[10:13]

    print('\n')
    print('year_list:', year_list)
    print('\n')
    print('onclick_id:', onclick_id)
    print('\n')
    
    print("==== processing by years ====")
    source_dict = {}
    
    # create web object
    driver = webdriver.Chrome()
    driver.get(url)
    
    for i in range(len(year_list)):
        # click to expand archive
        if onclick_id[i]=="2014":
            time.sleep(10)
            driver.find_element(By.ID, "divArchiveMain").click()
            
        source_dict[year_list[i]] = get_page(driver,onclick_id[i])
        print(onclick_id[i]+":",year_list[i],"finished")
    
    np.save(r'C://Users//raina//Dropbox//Chengjun//Text Analysis//yearly_page_source.npy',source_dict)
    return source_dict

In [173]:
url = "https://www.rbi.org.in/scripts/Annualpolicy.aspx"
source_dict = get_all_page(url)

<Response [200]>


year_list: ['2023-2024', '2022-2023', '2021-2022', '2020-2021', '2019-2020', '2018-2019', '2017-2018', '2016-2017', '2015-2016', '2014-2015', '2013-2014', '2012-2013', '2011-2012', '2010-2011', '2009-2010', '2008-2009', '2007-2008', '2006-2007', '2005-2006', '2004-2005', '2003-2004', '2002-2003', '2001-2002', '2000-2001', '1999-2000', '1936-1999']


onclick_id: ['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000', '1999']


==== processing by years ====
2024: 2023-2024 finished
2023: 2022-2023 finished
2022: 2021-2022 finished
2021: 2020-2021 finished
2020: 2019-2020 finished
2019: 2018-2019 finished
2018: 2017-2018 finished
2017: 2016-2017 finished
2016: 2015-2016 finished
2015: 2014-2015 finished
2014: 2013-2014 finished
2013: 2012-2013 finished
2012: 2011-2012 finished
2011: 2010-2011 finished
2010: 2009-2010 finished
2